# Lab03 - Box Modeling - Lab Bench

Load relevant packages with the code below and check to make sure you are working in the right directory.

In [ ]:
# Imports used throughout the lab (arrays, plotting, data frames, ODE integration)
from pathlib import Path

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.integrate import solve_ivp
from datetime import datetime

notebook_loc = Path(os.getcwd())
# check that has the right parent
if notebook_loc.parent.name != 'work':
  print('Please make a copy of this notebook and put it in your "work" folder!')
else:
  fig_dir = notebook_loc/'figures'
  fig_dir.mkdir(parents=True, exist_ok=True)
  print('Great, your figures will be saved in:', fig_dir)

def fig_save(fig_name, filetype='png'):
  fig_path = fig_dir/f'{fig_name}.{filetype}'
  unique_fig_path = fig_dir/f'{fig_path.name}__{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}{fig_path.suffix}'
  plt.savefig(fig_path, dpi=300)
  plt.savefig(unique_fig_path, dpi=300)

## Investigation

### What is a box model?

A box model represents a complex system as a set of reservoirs that exchange material through fluxes. Each box contains some quantity - here, carbon - and the model tracks how that quantity changes over time.

This simplification removes spatial detail but preserves the balances that control system behavior: how much carbon is stored, how quickly it moves, and where it can enter or leave the system.

In the context of the carbon cycle, this approach lets us study interactions between the atmosphere and the ocean without a full circulation model. The goal is not to reproduce geography, but to understand exchange, storage, and adjustment timescales.

In this lab, we will build and progressively modify a two-box model that represents the atmosphere and the surface ocean. Later in the course you will see how adding a deep-ocean box changes the system’s long-term storage and stability.

------------------------------------------------------------------------

### 2-Box Model (Atmosphere and Surface Ocean): Overview

#### Reservoirs

-   $A(t)$: Atmosphere box concentration (or anomaly)  
-   $S(t)$: Surface ocean box concentration (or anomaly)

As we set up the box model, we will initialize each reservoir with an initial condition. These initial values determine where the system begins in state space and how it evolves toward equilibrium.

#### Fluxes

If two boxes are connected, we must describe the flux from A -\> S and from S -\> A.

------------------------------------------------------------------------

### Closed System

In our first version of the box model, we will start with:

-   **Reservoirs:** Initial condition: \[800, 500\]  
-   **Flux:** Exchange governed by a rate constant $k$ (units: 1 / time)

The flux is proportional to the difference between the reservoirs:

-   Change in atmosphere:

    $$
    \frac{dA}{dt} = -k \, (A - S)
    $$

-   Change in surface ocean:

    $$
    \frac{dS}{dt} = k \, (A - S)
    $$

When $A > S$, carbon flows from the atmosphere into the surface ocean. The atmosphere loses carbon (negative change), and the surface ocean gains carbon (positive change). The flux magnitude scales with the concentration difference.

In a closed atmosphere-ocean system, carbon can move between reservoirs but cannot enter or leave the system as a whole. This constraint means total carbon must remain constant. The model may redistribute carbon, but it cannot create or destroy it. Equilibrium emerges internally as fluxes balance.

Define the two‑box system, helper functions, and plotting routines.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pandas as pd
import seaborn as sns

# Define the closed system ODE
def carbon_system(t, state, k):
    M_atm, M_surf = state

    # Conservation of mass equations
    dM_atm_dt = -k * (M_atm - M_surf)
    dM_surf_dt = k * (M_atm - M_surf)

    return [dM_atm_dt, dM_surf_dt]

The code below is a helper function that runs the model, calculates the derivative at each timestep, and constructs an ocean_df. As a reminder, the derivatives were simply calculated by approximately:

        dy0dt, dy1dt = [], []

        for ik, t in enumerate(sol.t):
            state = (sol.y[0][ik], sol.y[1][ik])
            dy0, dy1 = carbon_system(t, state, *args)
            dy0dt.append(dy0)
            dy1dt.append(dy1)

In [ ]:
def calc_C_dC(carbon_system, t_span, initial_conditions, args, timesteps=200,
             box_labels=("Atmosphere", "Surface Ocean")):

    arg_keys = ['k', 'R', 'l_s', 'buffer']

    t_eval = np.linspace(min(t_span), max(t_span), timesteps)
    sol = solve_ivp(carbon_system, t_span, initial_conditions, dense_output=True, args=args, t_eval=t_eval)
    derivative = np.array([carbon_system(t, y, *args) for t, y in zip(sol.t, sol.y.T)])

    atm_df = pd.DataFrame({'t': sol.t, 'C': sol.y[0], 'dC': derivative[:, 0]})
    atm_df['Box'] = box_labels[0]

    surf_df = pd.DataFrame({'t': sol.t, 'C': sol.y[1], 'dC': derivative[:, 1]})
    surf_df['Box'] = box_labels[1]

    ocean_df = pd.concat([atm_df, surf_df])
    ocean_df['init_cond'] = [tuple(initial_conditions) for _ in range(len(ocean_df))]

    for arg_key in arg_keys:
        ocean_df[arg_key] = 0

    for ip, arg in enumerate(args):
        ocean_df[arg_keys[ip]] = arg

    return ocean_df

The code cell below initalizes a plotting helper to produce the multiplots you will look at throughout the lab. (4 plot combination: time series, dC/dt, phase portrait, and state space)

In [ ]:
def plot_frames(_ocean_df, ax_row=None, nplots=2,
                box_labels=("Atmosphere", "Surface Ocean"),
                row_label=None,
                nullcline_plotter=None,
                nullcline_kwargs=None,
                init_cond_marker_map=None,
                init_cond_color_map=None):
    if ax_row is None:
        fig, ax_row = plt.subplots(nrows=1, ncols=nplots)

    if nullcline_kwargs is None:
        nullcline_kwargs = {}

    # Base time series and dC/dt plots (color by box only)
    ax_row[0] = sns.lineplot(data=_ocean_df, x='t', y='C', hue='Box', alpha=.5, units='init_cond', estimator=None, ax=ax_row[0], legend=False)
    ax_row[1] = sns.lineplot(data=_ocean_df, x='t', y='dC', hue='Box', alpha =.5, units='init_cond',estimator=None,  ax=ax_row[1], legend=False)

    # Phase portrait by box (C vs dC)
    ax_row[-2] = sns.lineplot(data=_ocean_df, x='C', y='dC', hue='Box', alpha =.5,units='init_cond', estimator=None, ax=ax_row[-2], legend=False)
    ax_row[-2].set(xlabel='C', ylabel='dC/dt')

    # State space (box vs box)
    atm = _ocean_df[_ocean_df['Box'] == box_labels[0]]
    surf = _ocean_df[_ocean_df['Box'] == box_labels[1]]

    tmp_df = pd.DataFrame({
        box_labels[0]: atm['C'].values,
        box_labels[1]: surf['C'].values,
        'init_cond': surf['init_cond'].values,
        't': surf['t'].values,
    })
    tmp_df.init_cond = tmp_df.init_cond.astype('category')
    ax_row[-1] = sns.lineplot(data=tmp_df, x=box_labels[0], y=box_labels[1], units='init_cond',estimator=None,  ax=ax_row[-1], legend=False)
    ax_row[-1].set(xlabel=f'{box_labels[0]} C', ylabel=f'{box_labels[1]} C')

    # Row label in first plot ylabel
    if row_label is not None:
        ax_row[0].set_ylabel(f'C ({row_label})')
    else:
        ax_row[0].set_ylabel('C')

    # Marker styles for initial conditions
    unique_inits = list(tmp_df['init_cond'].cat.categories)
    if init_cond_marker_map is None:
        marker_cycle = ['o', 's', '^', 'D', 'P', 'X']
        init_cond_marker_map = {ic: marker_cycle[i % len(marker_cycle)] for i, ic in enumerate(unique_inits)}

    if init_cond_color_map is None:
        init_colors = sns.color_palette(n_colors=len(unique_inits))
        init_cond_color_map = {ic: init_colors[i] for i, ic in enumerate(unique_inits)}

    # Colors by box
    box_colors = dict(zip(box_labels, sns.color_palette(n_colors=len(box_labels))))

    # Add start (filled) and end (open) points in plots 3 and 4
    for init_cond in unique_inits:
        marker = init_cond_marker_map[init_cond]

        # For each box, grab start/end in C vs dC (plot 3) and color by box
        for box in box_labels:
        # for grp, box_df in _ocean_df.groupby(['Box','init_cond' ]):
        #     box=grp[0]
        #     init_cond=grp[1]
            box_df = _ocean_df[(_ocean_df['Box'] == box) & (_ocean_df['init_cond'] == init_cond)]
            if box_df.empty:
                continue
            start = box_df[box_df['t']==box_df['t'].min()].iloc[0]
            end = box_df[box_df['t']==box_df['t'].max()].iloc[0]
            color = box_colors[box]

            ax_row[-2].scatter(start['C'], start['dC'], marker=marker,
                               facecolors=color, edgecolors=color, s=40, zorder=5)
            ax_row[-2].scatter(end['C'], end['dC'], marker=marker,
                               facecolors='none', edgecolors=color, s=40, zorder=5)

        # For the state-space plot (plot 4), color by initial condition
        ic_df = tmp_df[tmp_df['init_cond'] == init_cond]
        if ic_df.empty:
            continue
        start = ic_df[ic_df['t']==ic_df['t'].min()].iloc[0]
        end = ic_df[ic_df['t']==ic_df['t'].max()].iloc[0]

        # start = ic_df.iloc[0]
        # end = ic_df.iloc[-1]
        ic_color = init_cond_color_map[init_cond]
        ax_row[-1].scatter(start[box_labels[0]], start[box_labels[1]], marker=marker,
                           facecolors=ic_color, edgecolors=ic_color, s=40, zorder=5)
        ax_row[-1].scatter(end[box_labels[0]], end[box_labels[1]], marker=marker,
                           facecolors='none', edgecolors=ic_color, s=40, zorder=5)

    # Optional nullclines overlaid on state-space plot
    if nullcline_plotter is not None:
       ax_row[-1], c1, c2, h, l = nullcline_plotter(**nullcline_kwargs)
       print(l)

    # Legends: always show box colors. Only show init_cond if <4.
    # Box legend handles
    box_handles = [plt.Line2D([0], [0], color=box_colors[b], lw=2, label=b) for b in box_labels]

    legend_handles = box_handles
    legend_labels = [b for b in box_labels]

    if len(unique_inits) < 4:
        init_handles = [plt.Line2D([0], [0], marker=init_cond_marker_map[ic],
                                   color=init_cond_color_map[ic], lw=0, markersize=7, label=str(ic))
                        for ic in unique_inits]
        

        legend_handles += init_handles
        legend_labels += [str(ic) for ic in unique_inits]
    start_handle = plt.Line2D([0], [0], marker='o', color='black', lw=0,
                                  markerfacecolor='black', markersize=6, label='start')
    end_handle = plt.Line2D([0], [0], marker='o', color='black', lw=0,
                                markerfacecolor='none', markersize=6, label='end')
    legend_handles+=[start_handle, end_handle]
    legend_labels += ['start', 'end']
    ax_row[-1].legend(legend_handles, legend_labels, bbox_to_anchor=(1.4, 1))

And finally, a piece of code to add nullclines.

In [ ]:
def plot_nullclines(carbon_system_vec, ax, args,
                    Ma_lim=(0, 25), Ms_lim=(0, 25), n=401,
                    add_quiver=False, quiver_step=25, solo=False):
    # Plot nullclines in (Ma, Ms) space, plus (optional) vector field arrows.

    Ma = np.linspace(*Ma_lim, n)
    Ms = np.linspace(*Ms_lim, n)
    MA, MS = np.meshgrid(Ma, Ms)
    # carbon_system(t, state, k)
    dMA, dMS = carbon_system_vec(0, (MA, MS), *args)

    # Nullclines are the zero-level contours
    linewidth = 1
    if solo is True:
        linewidth = 2

    c1 = ax.contour(MA, MS, dMA, levels=[0], linewidths=linewidth, alpha=.5, colors='steelblue',)
    c2 = ax.contour(MA, MS, dMS, levels=[0], linewidths=linewidth, alpha=.5, linestyles="--", colors='orange',)

    # Label for legend (matplotlib contour objects are awkward in legends)
    # We'll create proxy artists:
    atm_proxy = plt.Line2D([0], [0], linewidth=2, color='steelblue', label="dM_atm/dt = 0")
    surf_proxy = plt.Line2D([0], [0], linewidth=2, linestyle="--", color='orange', label="dM_surf/dt = 0")

    if add_quiver:
        # subsample for readable arrows
        sl = slice(None, None, quiver_step)
        ax.quiver(MA[sl, sl], MS[sl, sl], dMA[sl, sl], dMS[sl, sl],alpha=.5,
                  angles="xy", scale_units="xy", scale=1)

    if solo is True:
        ax.set_xlabel("M_atm")
        ax.set_ylabel("M_surf")
        ax.set_title(f"Nullclines (buffer={buffer})")

    # ax.legend([atm_proxy, surf_proxy], ["dM_atm/dt = 0", "dM_surf/dt = 0"])

    ax.set_xlim(Ma_lim)
    ax.set_ylim(Ms_lim)

    return ax, c1, c2, [atm_proxy, surf_proxy], ["dM_atm/dt = 0", "dM_surf/dt = 0"]


def get_lims(df, box):
    return (df[df['Box']==box]['C'].min(), df[df['Box']==box]['C'].max())

Let’s set a few parameters for the first interaction

In [ ]:
t_span = (0, 50) # 50 years
initial_conditions = [800, 500] # [Atmosphere, Surface Ocean]
k_rate = 0.1 # Exchange coefficient

In [ ]:
args = (k_rate,)

ocean_df = calc_C_dC(carbon_system, t_span, initial_conditions, args, timesteps=400)

# Uncomment to compare with a second initial condition
# ocean_df2 = calc_C_dC(carbon_system, t_span, (100, 600), args, timesteps=200)
# ocean_df = pd.concat([ocean_df, ocean_df2])

fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(20, 4))
plot_frames(ocean_df, ax_row=axs, nullcline_plotter=plot_nullclines, nullcline_kwargs=
                    dict(carbon_system_vec=carbon_system, ax=axs[-1], args=args,
                    Ma_lim=get_lims(ocean_df, 'Atmosphere'), Ms_lim=get_lims(ocean_df, 'Surface Ocean'), n=401,
                    add_quiver=True, quiver_step=25, solo=False))

# fig_save('closed_system_init_cond')

Time series show how each reservoir evolves as the system moves toward equilibrium. These trajectories reflect flux imbalance: large imbalances produce rapid change; small imbalances produce slow adjustment.

From left to right, the panels show:

1.  carbon in each reservoir over time  
2.  rate of change over time  
3.  rate of change as a function of the reservoir carbon (a 1D phase portrait for each box)  
4.  atmosphere carbon vs surface ocean carbon at each timestep (a 2D state-space trajectory)

Look at the plots, then uncomment the commented code above and rerun it.

------------------------------------------------------------------------

### Adding a Source

When carbon is added from outside the system - for example, through fossil fuel emissions - equilibrium is no longer determined solely by internal flux balance.

Now we add a source term - for example, an emissions flux - into the atmosphere.

$$
\frac{dA}{dt} = -k \, (A - S) + R
$$

Here, $R$ represents an external carbon input rate.

Because carbon is now entering the system but not leaving it, total carbon is no longer conserved. Instead, total carbon increases at rate $R$:

$$
\frac{d}{dt}(A + S) = R
$$

Now add the emissions term, `R`, to the `carbon_system_R` function, the way you see it described above, and set it to `.3`. (You can try other values later, if you like.)

In [ ]:
def carbon_system_R(t, state, k, R):
    M_atm, M_surf = state

    # Updated mass equations
    dM_atm_dt = -k * (M_atm - M_surf) + 
    dM_surf_dt = k * (M_atm - M_surf)

    return [dM_atm_dt, dM_surf_dt]

Set the value for `R`

In [ ]:
t_span = (0, 200) # 200 years
initial_conditions = [0, 0] # [Atmosphere, Surface Ocean]
R = 

Run the code below to look at your new model.

In [ ]:
args = (k_rate, R, )

ocean_df_R = calc_C_dC(carbon_system_R, t_span, initial_conditions, args, timesteps=200)

## Uncomment to compare with a second initial condition
# ocean_df2_R = calc_C_dC(carbon_system_R, t_span, (5, 15), args, timesteps=200)
# ocean_df_R = pd.concat([ocean_df_R, ocean_df2_R])

fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(20, 4))
plot_frames(ocean_df_R, ax_row=axs, nplots=2)
# _ = plot_nullclines(carbon_system_R, axs[-1], args,
#                     Ma_lim=get_lims(ocean_df_R, 'Atmosphere'), Ms_lim=get_lims(ocean_df_R, 'Surface Ocean'), n=401,
#                     add_quiver=True, quiver_step=25, solo=False)

plot_frames(ocean_df_R, ax_row=axs, nullcline_plotter=plot_nullclines, nullcline_kwargs=
                    dict(carbon_system_vec=carbon_system_R, ax=axs[-1], args=args,
                    Ma_lim=get_lims(ocean_df_R, 'Atmosphere'), Ms_lim=get_lims(ocean_df_R, 'Surface Ocean'), n=401,
                    add_quiver=True, quiver_step=25, solo=False))                    

# fig_save(source_init_cond)

Now uncomment the commented code above and rerun the model.

------------------------------------------------------------------------

### Concentration Driven Sink

To balance the source term introduced previously, we now add a sink term.

There are several ways to do this. Previously, we added a constant source. Add a sink to `carbon_system_R_l` (below) that is proportional to the amount present by including a `ls_s*M_surf` term in the `dM_surf_dt` equation.

In [ ]:
def carbon_system_R_l(t, state, k, R, l_s):
    M_atm, M_surf = state

    # Mass equations with source and sink (export out of surface box)
    dM_atm_dt = -k * (M_atm - M_surf) + R
    dM_surf_dt = k * (M_atm - M_surf) - #sink term

    return [dM_atm_dt, dM_surf_dt]

Set `l_s` to `.2`

In [ ]:
t_span = (0, 100) # 50 years
initial_conditions = [0, 0] # [Atmosphere, Surface Ocean]
l_s = 

Then run your model:

In [ ]:
args = (k_rate, R, l_s, )

ocean_df_R_B = calc_C_dC(carbon_system_R_l, t_span, initial_conditions, args, timesteps=200)

fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(20, 4))
plot_frames(ocean_df_R_B, ax_row=axs, nullcline_plotter=plot_nullclines, nullcline_kwargs=
                    dict(carbon_system_vec=carbon_system_R_l, ax=axs[-1], args=args,
                    Ma_lim=get_lims(ocean_df_R_B, 'Atmosphere'), Ms_lim=get_lims(ocean_df_R_B, 'Surface Ocean'), n=401,
                    add_quiver=True, quiver_step=25, solo=False))


Now let’s suppose we don’t start with empty reservoirs. The code below loops through a bunch of initial conditions.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(20, 4))

ocean_df_R_Bs = []
for init_cond_a in [0, 5, 10, 20]:
    for init_cond_s in [0, 5, 10, 20]:
        init_cond = (init_cond_a, init_cond_s)

        ocean_df_R_B = calc_C_dC(carbon_system_R_l, t_span, init_cond, args, timesteps=200)
        ocean_df_R_Bs.append(ocean_df_R_B)

ocean_df_R_B = pd.concat(ocean_df_R_Bs)
plot_frames(ocean_df_R_B, ax_row=axs, nullcline_plotter=plot_nullclines, nullcline_kwargs=
                    dict(carbon_system_vec=carbon_system_R_l, ax=axs[-1], args=args,
                    Ma_lim=get_lims(ocean_df_R_B, 'Atmosphere'), Ms_lim=get_lims(ocean_df_R_B, 'Surface Ocean'), n=401,
                    add_quiver=True, quiver_step=25, solo=False))

# fig_save(sink_init_cond)

The system now has competing processes: input, removal, and redistribution. Equilibrium is no longer determined by internal balance alone, nor does it drift indefinitely as in the source-only case. Instead, steady state emerges from the balance of external forcing and internal dynamics. This places the model closer to real carbon cycle behavior, where multiple processes operate simultaneously.

------------------------------------------------------------------------

## Nonlinear Flux (Buffer Chemistry)

Previously, flux depended directly on reservoir concentration. Here, we represent buffering by defining an effective concentration that mediates exchange:

$$
M_{ ext{eff}} = \frac{M}{1 + \beta M}
$$

Fluxes are then calculated using $M_{\text{eff}}$ rather than the raw concentration $M$.

This time the adjustment to the system is already in place.

In [ ]:
def carbon_system_R_l_b(t, state, k, R, l_s, buffer):
    M_atm, M_surf = state

    M_surf_eff = M_surf / (1 + buffer * M_surf)

    # Conservation of mass equations
    dM_atm_dt = -k * (M_atm - M_surf_eff) + R
    dM_surf_dt = k * (M_atm - M_surf_eff) - l_s * M_surf_eff

    return [dM_atm_dt, dM_surf_dt]

Running the code below will produce a figure with two rows: (top) no buffer, (bottom) buffered flux. For the buffered run, `t_span` runs to 5000 instead of 200, as in the unbuffered case.

In [ ]:
buffers = [0, .2]

fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20,4*len(buffers)))

for ib, buffer in enumerate(buffers):
    ax_row = axs[ib]
    buffer_dfs = []
    args = (k_rate, R, l_s, buffer)
    if buffer==0:
        t_span = (0, 200)
        timestep=1000
    else:
        t_span = (0, 5000)
        timestep=5000
    for init_cond_a in [0, 5, 10, 20, 50]:
        for init_cond_s in [0, 5, 10, 20]:
            init_cond = (init_cond_a, init_cond_s)

            ocean_df_R_B_buffer = calc_C_dC(carbon_system_R_l_b, t_span, init_cond, args, timesteps=timestep)
            buffer_dfs.append(ocean_df_R_B_buffer)

    ocean_df_R_B_buffer = pd.concat(buffer_dfs)
    # plot_frames(ocean_df_R_B_buffer, ax_row=ax_row, nplots=2)
    # plot_nullclines(carbon_system_R_l_b, ax_row[-1], args,
    #                 Ma_lim=get_lims(ocean_df_R_B_buffer, 'Atmosphere'), Ms_lim=get_lims(ocean_df_R_B_buffer, 'Surface Ocean'), n=401,
    #                 add_quiver=True, quiver_step=25, solo=False)
    plot_frames(ocean_df_R_B_buffer, ax_row=axs, nullcline_plotter=plot_nullclines, nullcline_kwargs=
                    dict(carbon_system_vec=carbon_system_R_l_b, ax=axs[-1], args=args,
                    Ma_lim=get_lims(ocean_df_R_B_buffer, 'Atmosphere'), Ms_lim=get_lims(ocean_df_R_B_buffer, 'Surface Ocean'), n=401,
                    add_quiver=True, quiver_step=25, solo=False))

# fig_save(nonlin_flux)